# **Deteccion de fraude crediticio con machine learning** (Ricardo Bustos Carreón)
Partimos de un conjunto de datos con 7300 entradas y 31 variables de las cuales 29 son desconocidas, dos conocidas y el tipo de variable. De estas entradas 7000 son transacciones normales y 300 son transacciones fraudulentas, nuestro proposito es aprender a identificar si una operacion es legal o fraudulenta dado que este tipo de operaciones cuestan tanto al cliente como a la empresa, por eso comenzamos con un analisis enfocado el machine learning.

## **Analisis exploratorio de datos**
Ya conocemos nuestras variables, procederemos al analisis. No podemos basar si una transaccion es fraudulenta en funcion de la media o maxima pues la distribución del valor monetario de todas las transacciones está muy sesgada. La gran mayoría de las transacciones son relativamente pequeñas y solo una pequeña fracción de las transacciones se acerca al máximo.

Ahora, ¿Qué pasa con las distribuciones de clase? ¿Cuántas transacciones son fraudulentas y cuántas no lo son? Bueno, como podemos esperar, la mayoría de las transacciones no son fraudulentas (95.890%), mientras que solo el 4.110% fueron fraudulentas.
Visualicemoslo ahora con nuestro algoritmo: 


In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

df = pd.read_csv('C:/Users/ricar/Desktop/dataset.csv')
print('El data frame tiene {} filas y {} columnas.'.format(df.shape[0], df.shape[1]))
print (df.sample(5))
print(df.info())

df.loc[:, ['Var', 'Amount']].describe()
#visualizacions de Var y amount
plt.figure(figsize=(10,8))
plt.title('Distribucion de la variable (Var)')
sns.distplot(df.Var)


El data frame tiene 7300 filas y 31 columnas.
         Var        V1        V2        V3  ...       V27       V28  Amount  Class
1652  105397  1.287631  0.410746  0.151841  ...  0.030363  0.020647    6.95      0
4902  129463  1.454601 -1.063245  0.440964  ...  0.021222  0.011972   25.00      0
2568   15877  1.219136  0.535374 -0.482669  ... -0.041098  0.032509    0.76      0
947   169560 -1.317357  1.609457 -1.498172  ...  0.273136 -0.035843    8.97      0
839   233197  2.113497  0.575076 -2.803749  ...  0.023942 -0.002685    1.00      0

[5 rows x 31 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7300 entries, 0 to 7299
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Var     7300 non-null   int64  
 1   V1      7300 non-null   float64
 2   V2      7300 non-null   float64
 3   V3      7300 non-null   float64
 4   V4      7300 non-null   float64
 5   V5      7300 non-null   float64
 6   V6      7300 non-null   f

In [1]:
plt.figure(figsize=(10,8))
plt.title('Distribucion del valor monetario (Amount)')
sns.distplot(df.Amount)


<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/315073115_3266202986977681_5264328340428307278_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeHCI8zgQYdVlxaEPk_AKdCDOrmmh0a7JBA6uaaHRrskEDfKjKASta6857Q_LXz_f4a5VC5Ybgeq0s-UX0JQ-YaT&_nc_ohc=0cHt4AVji-oAX_jC2Z7&tn=iAnTb-G2baG76wTk&_nc_ht=scontent.fmex5-1.fna&oh=00_AfBkv1HGDFGEjdIcBsYaIPjchSRln8mzP1eV1sBzvovh9Q&oe=636F3A26' alt='python' width='400' height='400'>


In [1]:
#Comparacion de transacciones fraudulentas contra normales
counts = df.Class.value_counts()
normal = counts[0]
fraudulent = counts[1]
perc_normal = (normal/(normal+fraudulent))*100
perc_fraudulent = (fraudulent/(normal+fraudulent))*100
print('Tenemos {} transacciones normales ({:.3f}%) y {} transacciones fraudulentas ({:.3f}%).'.format(normal, perc_normal, fraudulent, perc_fraudulent))
plt.figure(figsize=(8,6))
sns.barplot(x=counts.index, y=counts)
plt.title('Transacciones normales contra fraudulentas')
plt.ylabel('Count')
plt.xlabel('Class (0:Normales, 1:Fraudulentas)')


Tenemos 7000 transacciones normales (95.890%) y 300 transacciones fraudulentas (4.110%).
<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/314777348_3266202960311017_8811628485310736382_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeFndORm5pymB2H0mDyIuESrA2ZO7l_wIfUDZk7uX_Ah9YlVZz5OkRnKA7W0bqm4SK-eXg7RWcaIKwTJ_JpFqn1v&_nc_ohc=C8ZaVes1tUAAX8_UmA4&_nc_ht=scontent.fmex5-1.fna&oh=00_AfA5Ww79myRSh-6MYw045ZgwkqAVp4j2yowZ2iPnPvnMvg&oe=636EA2DE' alt='python' width='400' height='400'>


Observamos ahora la correlacion entre nuestras variables, para este caso usamos un mapa de calor, podemos apreciar una fuerte correlacion entre nuestras variables y las variables de clase.
Visualicemos nuestro algoritmo: 


In [1]:
corr = df.corr()
print (corr)
#Mapa de calor para visualización
corr = df.corr()
plt.figure(figsize=(12,10))
heat = sns.heatmap(data=corr)
plt.title('Mapa de calor para correlacion')
#Simetria
skew_ = df.skew()
print (skew_)


             Var        V1        V2  ...       V28    Amount     Class
Var     1.000000  0.139439 -0.038180  ... -0.000669 -0.009194 -0.059608
V1      0.139439  1.000000 -0.326759  ... -0.056911 -0.208454 -0.391060
V2     -0.038180 -0.326759  1.000000  ... -0.052532 -0.501140  0.372244
V3     -0.185765  0.484560 -0.408068  ... -0.039208 -0.147528 -0.569067
V4     -0.125990 -0.279497  0.266135  ...  0.053758  0.095444  0.512064
V5      0.184853  0.457110 -0.290087  ... -0.048134 -0.298085 -0.380176
V6     -0.032212  0.089315 -0.135573  ...  0.062639  0.204001 -0.194888
V7      0.116759  0.486588 -0.470636  ...  0.072419  0.247992 -0.527325
V8     -0.069866 -0.071541  0.031140  ... -0.033491 -0.068391  0.093921
V9      0.050420  0.263985 -0.250592  ...  0.035151 -0.024197 -0.399792
V10     0.075346  0.420311 -0.359026  ...  0.020854 -0.070770 -0.622983
V11    -0.230156 -0.300984  0.282850  ...  0.028729  0.016035  0.559960
V12     0.130645  0.402597 -0.376569  ... -0.020242 -0.000617 -0

## **Algoritmo nivel 2**
### **Description:**
Your task is write a function which formats a duration, given as a number of seconds, in a human-friendly way.
The function must accept a non-negative integer. If it is zero, it just returns 'now'. Otherwise, the duration is expressed as a combination of years, days, hours, minutes and seconds.
For the purpose of this problem, a year is 365 days and a day is 24 hours.
Note that spaces are important.
### Detailed rules
The resulting expression is made of components like 4 seconds, 1 year, etc. In general, a positive integer and one of the valid units of time, separated by a space. The unit of time is used in plural if the integer is greater than 1.
The components are separated by a comma and a space (', '). Except the last component, which is separated by ' and ', just like it would be written in English.
A more significant units of time will occur before than a least significant one. Therefore, 1 second and 1 year is not correct, but 1 year and 1 second is.
Different components have different unit of times. So there is not repeated units like in 5 seconds and 1 second.
A component will not appear at all if its value happens to be zero. Hence, 1 minute and 0 seconds is not valid, but it should be just 1 minute.
A unit of time must be used 'as much as possible'. It means that the function should not return 61 seconds, but 1 minute and 1 second instead. Formally, the duration specified by of a component must not be greater than any valid more significant unit of time.


In [2]:
### Algoritmo nivel 2
import math #Libreria a usar
from math import * #lo que importaremos de la libreria
def time(n): 
    if n == 0:
        return 'now' #si el tiempo es igual a cero inmediatamente nos arroja el ahora
    else:
        diva = math.trunc(n/31536000) #divisivilidad por años
        n = n-diva*31536000 #Se modifica el valor de n
        divd = math.trunc(n/86400) #divisivilidad por dias
        n = n-divd*86400 #Se modifica el valor de n
        divh = math.trunc(n/3600) #divisivilidad por horas
        n = n-divh*3600 #Se modifica el valor de n
        divm = math.trunc(n/60) #divisivilidad por minutos
        divs = n-divm*60 #divisivilidad por segundos
        divlists = [diva,divd,divh,divm,divs] #En esta lista guardamos nuestras unidades de tiempo
        year = str(diva)+' years' if diva > 1 or diva < -1 else str(diva)+' year' #Convertimos a strings
        day = str(divd)+' days' if divd > 1 or divd < -1 else str(divd)+' day' #Convertimos a strings
        hour = str(divh)+' hours' if divh > 1 or divh < -1 else str(divh)+' hour' #Convertimos a strings
        minute = str(divm)+' minutes' if divm > 1 or divm < -1 else str(divm)+' minute' #Convertimos a strings
        second = str(divs)+' seconds' if divs > 1 or divs < -1 else str(divs)+' second' #Convertimos a strings
        tl = [year,day,hour,minute,second] #Lista de unidades de tiempo
        date = [tl[i] for i in range(len(divlists)) if divlists[i] != 0] #aqui seleccionamos solo a los que tienen tiempos distintos de cero
        predate = [', '.join(date[:len(date)-1]), date[-1]] #unimos en un string
        return ' and '.join(predate)

### Pruebas
print (time(31536000+7200+120+59))
print (time(62))
print (time(3662))
print (time(0))
print (time(-31536000-7200-120-59))
print (time(-62))
print (time(-3662))
print (time(-0))


1 year, 2 hours, 2 minutes and 59 seconds
1 minute and 2 seconds
1 hour, 1 minute and 2 seconds
now
-1 year, -2 hours, -2 minutes and -59 seconds
-1 minute and -2 seconds
-1 hour, -1 minute and -2 seconds
now


## **Algoritmo nivel 3**
### **Description:**
My friend John likes to go to the cinema. He can choose between system A and system B.
System A : he buys a ticket (15 dollars) every time
System B : he buys a card (500 dollars) and a first ticket for 0.90 times the ticket price, then for each additional ticket he pays 0.90 times the price paid for the previous ticket.
John wants to know how many times he must go to the cinema so that the final result of System B, when rounded up to the next dollar, will be cheaper than System A.
The function movie has 3 parameters: card (price of the card), ticket (normal price of ticket), perc (fraction of what he paid for the previous ticket) and returns the first n such that


In [2]:
### Algoritmo nivel 3

def movie(card, ticket, perc):
    SA = ticket
    SB = card#+ticket*(perc)
    count = 1
    while SB > SA:
        SA = ticket*count
        SB = SB+ticket*(perc**count)
        count+=1
    return 'You must go '+str(count-1)+' times to the cinema, with card the total price is '+str(round(SB))+', with tickets '+str(SA) #es count-1 porque el while suma 1 al final del ciclo

### Pruebas
print (movie(500,15,0.9))
print (movie(100,10,0.95))


You must go 43 times to the cinema, with card the total price is 634, with tickets 645
You must go 24 times to the cinema, with card the total price is 235, with tickets 240
